知识参考：

[1] [强化学习之二：Q-Learning原理及表与神经网络的实现（Q-Learning with Tables and Neural Networks）](https://blog.csdn.net/qq_32690999/article/details/78996381)

[2] [深度学习框架Keras学习系列（二）：神经网络与BP算法（Neural Network and BP Algorithm）](https://blog.csdn.net/qq_32690999/article/details/78605371)

[3] [TensorFlow API](https://www.tensorflow.org/versions/r1.11/api_docs/python/tf/reset_default_graph)

# 一、Q-Learning

Q(s,a)指的是对于“在状态s下，采取行动a”的一个回报/价值估计，我们根据Q值来决策agent的下一步行动，并在不断行动的过程中，利用贝尔曼方程来学习/更新Q值，以学习如何更好地表现/达到更好的效果。

- 贝尔曼方程

$$Q(s,a)=Reward+\gamma*(max_{a'} Q(s',a'))$$

意义：某一个状态与行动对应的Q值，等于当前回报+折现率*（采取下一行动a'后，到达状态s'的最大可能值）

- 由贝尔曼方程衍生出的Q值更新公式

借由贝尔曼方程得到的估计Q值的方法，我们采取“采样”（sample）的思想：**sample的本质就是在未知的情况下，去尝试获得碎片信息，然后一点点把碎片拼起来，就获得了完整的信息**。意即每当我们完成一次行动，就相当于做了一次sample：

$$sample=Q(s,a)=Reward+\gamma*(max_{a'} Q(s',a'))$$

然后，我们把根据此次sample获得的新的信息更新到旧的Q值之中（更新的幅度由学习率控制）：

$$Q(s,a) = Q(s,a) + lr*(r + y*max_a Q(s1,a) - Q(s,a))$$

意义：新的Q值=旧的Q值+学习率lr\*（回报+折现率\*当前状态s1下能获得的最大Q值-旧的Q值)

然后我们不断重复【行动-采样-更新】这个过程，直到agent的表现令人满意！


# 二、Neural Network Based Q-Learning

当s和a都是有限集合的时候，(s,a)这样一个状态+行动的组合是有限的，因此我们可以将所有的Q值都“记住”，比如用一个列为行动，行为状态的表存起来，然后每次更新时，直接更新表中对应的Q值即可。

|Q(s,a)|action-1|action-2|...|
|--|--|--|--|
|state-1|Q(state-1,action-1)|Q(state-1,action-2)|...|
|state-2|Q(state-2,action-1)|Q(state-2,action-2)|...|
|...|...|...|...|
                                    
但是，更一般的情况其实应该是状态s是一个非常大的集合或者无限集合。比如说，如果冰湖环境不再以方格（字符）作为单位，而是以显示屏像素作为移动单位，那么此时s的可能情况就已经非常大了。

这意味着Q值表的尺寸也会非常大。Q表尺寸大会带来的最直接的问题就是：

-（1）你需要训练非常多的episodes以保证每个状态+动作的组合都被采样过足够多的次数，以保证学到足够完整正确的信息，而这样耗时是非常大的。
-（2）你需要很大的空间来存Q表
                                    
但是，我们依然需要对Q(s,a)值进行评估，那该怎么办呢？
                                    
## 基于神经网络（Neural Network）的Q-Learning方案

神经网络的input layer接受向量形式的输入，于是我们可以考虑将agent当前的状态以one-hot编码的形式输入到network中，然后output layer设置为对于各个行动的评估值向量。换言之，输出就是$[Q(s_i,a_1),Q(s_i,a_2),...,Q(s_i,a_n)]$


![pic1](NeuralNetworkBasedQLearning-pic1.png)              


通过训练，我们期望network能够尽可能准确输出对应状态下的所有Q值，并按照之前类似的做法，选择最大的Q值对应的行动来执行。

而训练network的具体算法依旧是**BP算法**，即误差逆传播（具体请参考*知识参考[2]*的内容，这里只做简单讲解），而误差就产生在：**对当前状态s的估值Q向量V1，与当前状态s下，根据策略选则采取行动a后到达的新状态s'下的Q值向量V2之间，在对行动a估值上的差。这也正是之前的基于表的Q-Learning利用贝尔曼方程针对性地更新Q(s,a)值时所用到的所有变量。**

***

- Example

举例来说，若在一个2*2的网格环境（（0，0），（0，1），（1，0），（1，1）四个方格）中，假设agent正处在(0,0)位置，即s=(0,0)。

那么我们首先将s=(0,0)编码成向量形式V_s=(1,0,0,0)输入到network，network会输出对于当前状态s的Q值向量：$$V_{qs}=(1.8,1.4,-1.2,0.2)$$，四个分量分别对应状态s下，【向上，向下，向左，向右】的各个行动的Q值；

之后，我们根据我们的行动策略，从V_qs选择Q值最大的那个行动a来执行，即执行【向上】；

网格环境反馈给我们reward=2以及下一状态s'=（0，1），那么我们此时再将新状态s'编码V_{s'}=(0,1,0,0)输入到network，并得到输出$$V_{qs}'=(2,5,0.5,1.2,0.2)$$；

根据Bellman Equation，此时，我们取最大的Qmax值=2.5，并进行处理:$$Q_{processed}=reward+\gamma * Qmax=2+\gamma * 2.5$$，假设$\gamma=0.8$，则Q_{processed}=2+2=4；

接着，我们就将network的目标向量设置为V_qs替换q(s,'向上')的值为Q_processed之后的那个向量，即：$$V_{goal}=(4,1.4,-1.2,0.2)$$；

现在，我们就可以计算估计出的向量$v_{qs}$与目标向量$v_{goal}$的差值，这与有监督学习中的network计算预测向量$\hat{y}与y$之间的差是一模一样的。

最后，按照一般network的误差前馈+梯度下降方式来训练network的参数即可，直到agent获得比较好的性能。

***

      
                                

## 超参数

- 学习率：$\gamma$

- 折现率：lr

- 随机行动概率：e



In [1]:
# Q-Network Learning on FrozenLake
# Q网络学习

import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

# 加载环境
env = gym.make('FrozenLake-v0')

# Q网络方法

# 架构：
  # 输入层：1*16 
  # 无隐藏层  
  # 输出层 16*4

# 初始化tf计算图
tf.reset_default_graph()

# 下面的几行代码建立了网络的前馈部分，它将用于选择行动
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32) # 定义输入向量；shape=[1,16]表示一个1*16的列向量，对应着冰湖环境中的16个方格/状态
W = tf.Variable(tf.random_uniform([16,4],0,0.01)) # 定义权重矩阵（此处输入向量是不断要输入新的值的，因此一般用placeholder来承载；而需要训练的参数如w,b都应该用Variable类来定义，因为Variable对象菜会在tf后续的训练中被优化）
Qout = tf.matmul(inputs1,W) # 输入向量*输入层到输出层的权重矩阵
predict = tf.argmax(Qout,1) # 预测将要选择的行动

# 下面的几行代码可以获得预测Q值与目标Q值间差值的平方和加总的损失。

nextQ = tf.placeholder(shape=[1,4],dtype=tf.float32) # 定义更新后的Q值向量
loss = tf.reduce_sum(tf.square(nextQ - Qout)) #计算差值向量的总和（损失函数的定义）
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1) # 用梯度下降来训练，学习率设置为0.1
updateModel = trainer.minimize(loss) # 训练模型（trianer最小化损失函数），minimize会自动更新Trainable的Variable类型的参数以最小化损失函数

# 训练网络
init = tf.initializers.global_variables() # 初始化全局参数

# 设置学习参数
y = .99  # 折现率
e = 0.1  # 随机行动的概率
num_episodes = 2000

# 创建列表以包含每个episode对应的总回报与总步数。
jList = []
rList = []

# 启动session；session是运行operation和对Tensor进行估值的必须环境
with tf.Session() as sess:
    sess.run(init) # 实际执行初始化
    for i in range(num_episodes):
        # 初始化环境并获得初始状态
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        
        # Q网络
        while j < 99:
            j+=1
            
            # 基于Q网络的输出结果，贪婪地选择一个行动（有一定的概率选择随机行动）
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:np.identity(16)[s:s+1]})
            
            # 基于随机数决定是否随机行动
            if np.random.rand(1) < e:
                a[0] = env.action_space.sample()

            # 获得行动后新的状态、回报、游戏是否结束等信息
            s1,r,d,_ = env.step(a[0])

            # 通过将新的状态向量输入到网络中获得新状态对应的Q值。
            Q1 = sess.run(Qout,feed_dict={inputs1:np.identity(16)[s1:s1+1]})

            # 获得最大的Q值
            # Recall: Q(s,a) = Q(s,a) + lr*(r + y*max_a(Q(s1,a)) - Q(s,a))
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + y*maxQ1
            

            # 用目标和预测的Q值训练网络
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[s:s+1],nextQ:targetQ})
            
            rAll += r
            s = s1
            if d == True:
                # 随着训练的进行，逐渐减少选择随机行为的概率
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)
print("成功得分的局数占比: " + str(sum(rList)/num_episodes) + "%")

成功得分的局数占比: 0.2595%


# 示例二：[CartPole-v0](https://gym.openai.com/envs/CartPole-v1/)

完成冰湖挑战后，我们再试试在经典的CartPole环境中实现NN-Based Q-Learning。

<video src="http://s3-us-west-2.amazonaws.com/rl-gym-doc/cartpole-no-reset.mp4" controls="controls">
</video>

比较典型、简单的Environment。

- Goal：左右移动黑色的长方块，保持连在长方块上面的棍子不倒下（只要棍子与垂直夹角达到15度，或者长方块左右移动距离超过2.4个单位，游戏就算结束）。
- Reward：每次行动后/每个时间戳，若棍子依然比较竖直（与垂线不超过15度），就获得+1分奖励。
- Actions：左，右
- Observation：
```
        Type: Box(4)
        Num	Observation                 Min         Max
        0	Cart Position             -4.8            4.8
        1	Cart Velocity             -Inf            Inf
        2	Pole Angle                 -24°           24°
        3	Pole Velocity At Tip      -Inf            Inf
```


In [7]:
# Q-Network Learning on CartPole-v0
# Q网络学习

import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

# 加载环境
env = gym.make('CartPole-v0')


# Q网络方法

# 架构：
  # 输入层：1*4 
  # 无隐藏层  
  # 输出层 4*2
    
state_variable_num=4; # cart position, cart velocity, pole angle, pole velocity at tip
action_num=2; # left, right

# 初始化tf计算图
tf.reset_default_graph()

# 下面的几行代码建立了网络的前馈部分，它将用于选择行动
inputs1 = tf.placeholder(shape=[1,state_variable_num],dtype=tf.float32) # 定义输入向量；shape=[1,16]表示一个1*16的列向量，对应着冰湖环境中的16个方格/状态
W = tf.Variable(tf.random_uniform([state_variable_num,action_num],0,0.01)) # 定义权重矩阵（此处输入向量是不断要输入新的值的，因此一般用placeholder来承载；而需要训练的参数如w,b都应该用Variable类来定义，因为Variable对象菜会在tf后续的训练中被优化）
Qout = tf.matmul(inputs1,W) # 输入向量*输入层到输出层的权重矩阵
predict = tf.argmax(Qout,1) # 预测将要选择的行动

# 下面的几行代码可以获得预测Q值与目标Q值间差值的平方和加总的损失。

nextQ = tf.placeholder(shape=[1,action_num],dtype=tf.float32) # 定义更新后的Q值向量
loss = tf.reduce_sum(tf.square(nextQ - Qout)) #计算差值向量的总和（损失函数的定义）
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1) # 用梯度下降来训练，学习率设置为0.1
updateModel = trainer.minimize(loss) # 训练模型（trianer最小化损失函数），minimize会自动更新Trainable的Variable类型的参数以最小化损失函数

# 训练网络
init = tf.initializers.global_variables() # 初始化全局参数

# 设置学习参数
y = .99  # 折现率
e = 0.1  # 随机行动的概率
num_episodes = 10

# 创建列表以包含每个episode对应的总回报与总步数。
jList = []
rList = []

# 启动session；session是运行operation和对Tensor进行估值的必须环境
with tf.Session() as sess:
    sess.run(init) # 实际执行初始化
    for i in range(num_episodes):
        # 初始化环境并获得初始状态
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        
        # Q网络
        while j < 99:
            j+=1
            
            # 基于Q网络的输出结果，贪婪地选择一个行动（有一定的概率选择随机行动）
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:np.reshape(s,[1,state_variable_num])}) # 状态不再用one-hot编码，而是直接用四个observation variable的值
            
            # 基于随机数决定是否随机行动
            if np.random.rand(1) < e:
                a[0] = env.action_space.sample()

            # 获得行动后新的状态、回报、游戏是否结束等信息
            s1,r,d,_ = env.step(a[0])

            # 通过将新的状态向量输入到网络中获得新状态对应的Q值。
            Q1 = sess.run(Qout,feed_dict={inputs1:np.reshape(s1,[1,state_variable_num])})

            # 获得最大的Q值
            # Recall: Q(s,a) = Q(s,a) + lr*(r + y*max_a(Q(s1,a)) - Q(s,a))
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + y*maxQ1
            

            # 用目标和预测的Q值训练网络
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.reshape(s,[1,state_variable_num]),nextQ:targetQ})
            
            rAll += r
            s = s1
            if d == True:
                # 随着训练的进行，逐渐减少选择随机行为的概率
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)
        
print('学到的权重矩阵W:')
print(W1)
print(rList)
print("成功得分的局数占比: " + str(sum(rList)/num_episodes) + "%")

学到的权重矩阵W:
[[-0.26104361 -1.12359834]
 [ 2.44668269 -4.98247766]
 [-0.01104815 -1.08332729]
 [-4.10341358  1.17452812]]
[75.0, 99.0, 64.0, 96.0, 84.0, 63.0, 52.0, 87.0, 49.0, 92.0]
成功得分的局数占比: 76.1%


In [8]:
# 测试不同agent的性能
def testAgent(W1=None,test_episodes=30):
    jList = []
    rList = []
    
    trained=True
    # random agent
    if W1 is None:
        trained=False
        
    for i in range(test_episodes):
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        
        while j<99:

            j+=1
            
            if trained:
                # 基于Network贪婪地选择一个最优行动（这里去掉了之前训练过程中加入的噪音干扰）
#                 print(np.dot(np.reshape(s,[1,4]),W1))
                a = np.argmax(np.dot(np.reshape(s,[1,4]),W1))

            else:
                a=env.action_space.sample()
            
            s,r,d,_=env.step(a)
            
            rAll+=r;
            
            # 判断游戏是否已经结束
            if d == True:
                break
        
        jList.append(j)
        rList.append(rAll)
    return jList,rList
print('基于学习到的Q值表，测试agent的性能：')
print()

print('随机行动/未经训练的agent平均得分：')
randomAgentResult=testAgent()
print(randomAgentResult[1])
print(sum(randomAgentResult[1])/len(randomAgentResult[1]))


print('Q-Learning agent平均得分：')
QLearningAgentResult=testAgent(W1)
print(QLearningAgentResult[1])
print(sum(QLearningAgentResult[1])/len(QLearningAgentResult[1]))


基于学习到的Q值表，测试agent的性能：

随机行动/未经训练的agent平均得分：
[14.0, 15.0, 13.0, 41.0, 36.0, 23.0, 21.0, 17.0, 13.0, 18.0, 14.0, 16.0, 12.0, 31.0, 93.0, 13.0, 18.0, 49.0, 18.0, 36.0, 12.0, 16.0, 19.0, 16.0, 17.0, 20.0, 20.0, 27.0, 18.0, 11.0]
22.9
Q-Learning agent平均得分：
[55.0, 59.0, 44.0, 47.0, 56.0, 42.0, 99.0, 74.0, 47.0, 99.0, 70.0, 51.0, 47.0, 99.0, 80.0, 89.0, 42.0, 42.0, 71.0, 53.0, 53.0, 48.0, 48.0, 40.0, 72.0, 37.0, 99.0, 56.0, 77.0, 66.0]
62.06666666666667
